In [2]:
import pandas as pd
import time 
import geocoder
import numpy as np

In [3]:
df=pd.read_csv('E:\Project\CLI_BB\output_file.csv')

In [44]:
df.shape

(19, 13)

In [4]:
df.head()

,Email,First Name,Last Name,Residential Address Street,Residential Address Locality,Residential Address State,Residential Address Postcode,Postal Address Street,Postal Address Locality,Postal Address State,Postal Address Postcode,Postal_co-ordinates,Residential_co-ordinates
0,colton_tromp@gmail.com,Darcy,Waters,8540 Charli Summit,AIRLIE BEACH,QLD,4802,376 Williamson Hill,ARTHUR RIVER,WA,6315,"(-20.272494000000002, 148.71814657732955)","(-33.3441135, 117.01571419732676)"
1,lindsay_bradtke@gmail.com,Georgia,Stewart,650 Harris Summit,ARCTURUS,QLD,4722,477 Jasmine Circle,ALDINGA,SA,5173,"(-23.975732999999998, 148.32711448169016)","(-35.2673234, 138.4831296)"
2,jaime_kuphal@yahoo.com,Joshua,Schmidt,53 Klein Circle,AMBLESIDE,TAS,7310,2979 Anthony Drive,AMOR,VIC,3825,"(-41.2050163, 146.3748003)","(-37.939795000000004, 146.4046230412041)"
3,odell.king@hotmail.com,Natalie,Smith,545 Summer Avenue,ALVIE,VIC,3249,5454 Koss Track,ANABRANCH SOUTH,NSW,2648,"(-38.2441811, 143.5155992)","(-33.6929151, 141.7057254)"
4,carlos@hotmail.com,Cameron,Schneider,46 Becker Summit,ALLANSFORD,VIC,3277,3137 Lakin Station St,ALLANS FLAT,VIC,3691,"(-38.3862028, 142.6001551)","(-36.2944155, 146.94292341122937)"


In [ ]:
df.dropna(inplace=True)

df['Residential Address Postcode'] = df['Residential Address Postcode'].astype(str)
df['Postal Address Postcode'] = df['Postal Address Postcode'].astype(str)
df['Postal Address Postcode'] = df['Postal Address Postcode'].str.replace('.0', '')

In [ ]:
df['Residential Address Full']=df['Residential Address Locality'] + ', ' + df['Residential Address State'] + ', ' + df['Residential Address Postcode']

df['Postal Address Full']=df['Postal Address Locality'] + ', ' + df['Postal Address State'] + ', ' + df['Postal Address Postcode']

In [ ]:
batch_size = 1000
batches = [df['Residential Address Full'][i:i+batch_size] for i in range(0, len(df), batch_size)]

In [ ]:
def geocode_address(address):
    num_retries = 0
    retry=3
    while num_retries < retry:
        location = geocoder.osm(address)
        try:
            if location.ok:
                return (location.lat, location.lng)
            else:
                print(f"Failed to geocode address: {address}")
                return np.nan
        except Exception as e:
            num_retries += 1
            time.sleep(0.5)
            continue 
    print(f'failed after retry for this add, {address}')
    return np.nan


In [ ]:
results = []
for i, batch in enumerate(batches):
    batch_results = [geocode_address(address) for address in batch]
    results.extend(batch_results)


df['Residential co-ordinates'] = pd.Series(results)
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
df.to_csv('updated_file_clients.csv', index=False)